https://colab.research.google.com/drive/1v114UId9huKyqAEU__aYuTMBFcMQghj9?usp=sharing

# Import Libraries needed

In [1]:
import pandas as pd
import numpy as np
!pip install git+git://github.com/altair-viz/altair.git
import altair as alt
from IPython import display
%matplotlib inline
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from keras import backend as K
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout, Flatten,Activation
from keras.layers.merge import dot
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K

  Cloning git://github.com/altair-viz/altair.git to /tmp/pip-req-build-qq1e5w2s
  Running command git clone -q git://github.com/altair-viz/altair.git /tmp/pip-req-build-qq1e5w2s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for altair: filename=altair-4.2.0.dev0-cp37-none-any.whl size=732213 sha256=01c7400f419a8a38c2fb93dbf80db366a1343e79207e4f8a736af35b8750d64a
  Stored in directory: /tmp/pip-ephem-wheel-cache-yczvy5oz/wheels/01/fd/91/025b6149b3949af76e93b3b3ceca5bf12cbdebc98fa46f9ec6
Successfully built altair
  Found existing installation: altair 4.1.0
    Uninstalling altair-4.1.0:
      Successfully uninstalled altair-4.1.0


Import Kaggle settings and download extra movie set

In [2]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
!kaggle datasets download -d rounakbanik/the-movies-dataset --unzip

Saving kaggle.json to kaggle.json
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              229KB  2021-05-21 06:23:33           5790  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           3219  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1140  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           1883  
promptcloud/careerbuilder-job-listing-2020

Retrieve u1.base data

In [3]:
df = pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u1.base",sep='\t',header=None, names=["userId", "movieId", "rating", "timestamp"])

Retrieve u1.test data

In [4]:
u1_test = pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u1.test",sep='\t',header=None, names=["userId", "movieId", "rating", "timestamp"])

Get metadata about movies

In [5]:
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [  'movieId', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols


movies=pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u.item",sep="|", names=movies_cols, encoding="latin-1'")

Get user information

In [6]:
users_cols = ['userId', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k//u.user", sep='|', names=users_cols, encoding='latin-1')

create merged matrix with all data so far

In [7]:
movielens =df.merge(movies, on='movieId').merge(users, on='userId')

#Data exploration

In [8]:
def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

def hist(field, label):
  return alt.Chart().mark_bar().encode(
      x=alt.X(field, bin=alt.Bin(maxbins=30), title=label),
      y="count()",
  ).properties(
      width=300,
  )
pd.DataFrame.flatten_cols = flatten_cols

In [9]:
movies_ratings = movies.merge(
    df
    .groupby('movieId', as_index=False)
    .agg({'rating': ['count', 'mean']})
    .flatten_cols(),
    on='movieId')

alt.hconcat(
    hist('rating count', '# ratings / movie'),
   hist('rating mean', 'mean movie rating'),
    data=movies_ratings)

alt.HConcatChart(...)

**ratings/ movie table** - the dataset has a lot of movies which have very little records 

**mean movie rating** - we can see a normal distribution for the movie ratings, the simple model will probably predict between 2.5 and 3.5 for movie ratings

In [10]:
users_ratings = users.merge(
    df
    .groupby('userId', as_index=False)
    .agg({'rating': ['count', 'mean']})
    .flatten_cols(),
    on='userId')

# Create a chart for the count, and one for the mean.
alt.hconcat(
    hist('rating count', '# ratings / user'),
    hist('rating mean', 'mean user rating'),
    data=users_ratings)

alt.HConcatChart(...)

**ratings / user** - we can see that your average user isn't leaving a ratings for movies they watch - it's likely that your average person is rating a movie either if they love the film or hate the film.

**mean user rating** - We can see a normal distirbution for user rating, givings us insight into why the mean movie rating is similiary distirbuted

#Correlation

In [11]:
movielens =df.merge(movies, on='movieId').merge(users, on='userId')
num = [item for item in range(0, 21)]
movielens['sex'].replace(['F','M'],[0,1],inplace=True)
movielens['occupation'].replace(movielens['occupation'].unique(),num,inplace=True)
movielens['Genre'] = movielens[movielens.columns[18:26]].apply(
    lambda x: ''.join(x.dropna().astype(str)),
    axis=1
)
movielens['Genre'] =movielens['Genre'].astype(str).astype(("category"))
movielens['Genre'] =movielens['Genre'].cat.codes
md_small = movielens.iloc[:, [2,28,29,31,27]]

We can see that there are linear relationships between the users age and rating, this makes sense as it's likely that as you get old your taste in movie change

In [12]:
c =md_small[md_small.columns[0:]].corr()
c.style.background_gradient(cmap='coolwarm')

,rating,sex,occupation,Genre,age
rating,1.000000,0.001150,-0.032682,0.011503,0.049143
sex,0.001150,1.000000,-0.097263,0.002778,0.013843
occupation,-0.032682,-0.097263,1.000000,-0.004288,-0.070139
Genre,0.011503,0.002778,-0.004288,1.000000,-0.004548
age,0.049143,0.013843,-0.070139,-0.004548,1.000000


I wanted to see if i could find a stronger correlation between genre and rating using non linear methods

In [13]:
c_n =md_small[md_small.columns[0:]].corr(method='spearman')
c_n.style.background_gradient(cmap='coolwarm')

,rating,sex,occupation,Genre,age
rating,1.000000,-0.004474,-0.024589,0.023554,0.039171
sex,-0.004474,1.000000,-0.105577,0.004034,0.002461
occupation,-0.024589,-0.105577,1.000000,-0.003172,-0.165109
Genre,0.023554,0.004034,-0.003172,1.000000,-0.007674
age,0.039171,0.002461,-0.165109,-0.007674,1.000000


#Baseline model for base RMSE

In [14]:
X = df.copy()
y = df['userId']
## split training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

In [15]:

## create pivot table to calculate mean easily
index=list(users['userId'].unique())
columns=list(movies['movieId'].unique())
index=sorted(index)
columns=sorted(columns)
util_df=pd.pivot_table(data=df,values='rating',index='userId',columns='movieId')
util_df.isnull().sum()


## get mean of movie using all users ratings and return it
def calculate_average_rating(movie_id):
   
   ratings_for_movie = util_df[movie_id]
   return ratings_for_movie.mean()

## calculate rmse score for baseline model   
def score_on_test_set():
  user_movie_pairs = zip(X_test['movieId'], X_test['userId'])
  predicted_ratings = np.array([calculate_average_rating(movie) for (movie,user) in user_movie_pairs])
  true_ratings = np.array(X_test['rating'])
  score = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
  return score
  

test_set_score = score_on_test_set()
print(test_set_score)

0.9913612360002064


We get an RMSE of 0.9913612360002064 when guessing the movie rating using the average movie rating for everyone who rated it.

#Simple model using Matrix Factorization

In [16]:
df_simple =  df.copy()
## convert user id to cat codes
df_simple['userId'] = df_simple['userId'].astype('category').cat.codes.values
df_simple['movieId'] = df_simple['movieId'].astype('category').cat.codes.values
## split data
train, test = train_test_split(df_simple, test_size=0.2)
## didnt use in the end
n_users, n_movies = len(df_simple['userId'].unique()), len( df_simple['movieId'].unique())
number_of_latent_factors = 64

In [17]:
import keras.backend as K



def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))
## create model
user_input=Input(shape=(2,),name='user_input',dtype='int64')
## create embedding for user
user_embedding=Embedding(2000,number_of_latent_factors,name='user_embedding')(user_input)
user_vec =Flatten(name='FlattenUsers')(user_embedding)
## create embedding for movie
movie_input=Input(shape=(2,),name='movie_input',dtype='int64')
movie_embedding=Embedding(2000,number_of_latent_factors,name='movie_embedding')(movie_input)
movie_vec=Flatten(name='FlattenMovies')(movie_embedding)
sim=euclidean_distance_loss(user_vec,movie_vec)
model = Model([user_input, movie_input],sim)

In [18]:
def rmse(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
model.compile(optimizer=Adam(lr=1e-4),loss='mse',metrics=[rmse])
batch_size=128
epochs=50


In [19]:
## fit matrix factorization model
model.fit([train.userId,train.movieId],train.rating, batch_size=batch_size,
                              epochs =epochs, validation_data = ([test.userId,test.movieId],test.rating),
                              verbose = 1)

Epoch 1/50
500/500 [==============================] - 3s 4ms/step - loss: 11.3023 - rmse: 3.3607 - val_loss: 10.5993 - val_rmse: 3.2544
Epoch 2/50
500/500 [==============================] - 2s 4ms/step - loss: 10.3341 - rmse: 3.2134 - val_loss: 9.7015 - val_rmse: 3.1135
Epoch 3/50
500/500 [==============================] - 2s 4ms/step - loss: 9.4537 - rmse: 3.0734 - val_loss: 8.8435 - val_rmse: 2.9725
Epoch 4/50
500/500 [==============================] - 2s 4ms/step - loss: 8.5651 - rmse: 2.9253 - val_loss: 8.0365 - val_rmse: 2.8335
Epoch 5/50
500/500 [==============================] - 2s 3ms/step - loss: 7.7696 - rmse: 2.7861 - val_loss: 7.2838 - val_rmse: 2.6975
Epoch 6/50
500/500 [==============================] - 2s 4ms/step - loss: 7.0025 - rmse: 2.6450 - val_loss: 6.5865 - val_rmse: 2.5650
Epoch 7/50
500/500 [==============================] - 2s 4ms/step - loss: 6.3168 - rmse: 2.5118 - val_loss: 5.9437 - val_rmse: 2.4365
Epoch 8/50
500/500 [==============================] - 2s 4m

In [20]:
model.evaluate([test.userId,test.movieId],test.rating)

500/500 [==============================] - 1s 2ms/step - loss: 1.0230 - rmse: 1.0040


[1.023004174232483, 1.0039982795715332]

Using matrix factorization to get a RMSE score of 1.0039982795715332

Using  u1.test to check rmse 

In [21]:
model.evaluate([u1_test.userId,u1_test.movieId],u1_test.rating)

625/625 [==============================] - 1s 2ms/step - loss: 2.0057 - rmse: 1.3825


[2.005692958831787, 1.3824903964996338]

RMSE of 1.3824903964996338 using test data

#Feature engineering: merge metadata

Use method to clean titles making it easier to merge data

In [22]:
## method to clean move title data so we can merge with original dataset
def clean_movie_title(movie_title):
    if movie_title.split(" ")[-1].startswith("("):
        movie_title = (" ".join(movie_title.split(" ")[:-1])).strip()
    if movie_title.title().split(',')[-1].strip() in ['The', 'A']:
        movie_title = (movie_title.title().split(',')[-1].strip() + " " + " ".join(movie_title.title().split(',')[:-1])).strip()
    movie_title = movie_title.lower()
    return movie_title

In [23]:
#read a clean version of the movie data
movies=pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u.item",sep="|", names=movies_cols, encoding="latin-1'")
movies = movies.drop(columns=["video_release_date"]) # wont be using this column
movies["title"] = movies["title"].apply(lambda row : clean_movie_title(row))  #clean movie titles for merge
movies["release_date"] = movies["release_date"].apply(lambda x : str(x).split("-")[-1]) # formate movie release date so i can merge with other data

## more data cleaning for merge
movies = movies[~movies.title.str.startswith("(")]
_df = movies[movies.title.str.contains("(", regex=False)]
_df.title = _df.title.apply(lambda x: re.search(r'\((.*?)\)', x).group(1).strip() if re.search(r'\((.*?)\)', x) else x.strip())
movies = movies.append(_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
# retrieve extra movie data
imdb_movie_keyword = pd.read_csv('/content/keywords.csv', encoding = "utf8",) 
imdb_movie_cast = pd.read_csv('/content/credits.csv',  encoding = "utf8") 
imdb_movie_meta_data = pd.read_csv('/content/movies_metadata.csv', encoding = "utf8") 

# merge new user data with movie data 
movielens_add =df.merge(movies, on='movieId').merge(users, on='userId')

# merging the miltiple data points keywords,credits,movies_metdata
# had to reformat parts of the dataset so i could merge
row = imdb_movie_meta_data.loc[imdb_movie_meta_data['id'] =='1997-08-20'].index[0]
imdb_movie_meta_data = imdb_movie_meta_data.drop(row)   ## weird some data added to id 
imdb_movie_meta_data['id'] = pd.to_numeric(imdb_movie_meta_data['id'] ,errors='coerce').dropna().astype("int64")
add = imdb_movie_keyword.merge(imdb_movie_meta_data,on="id")
## merge
merged_imdb = add.merge(imdb_movie_cast,on="id")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
### new dataset needs to be cleaned a bit more before merging with movielens
merged_imdb['title'] = merged_imdb['title'].astype(str) ## change to string column
merged_imdb["title"] = merged_imdb["title"].apply(lambda row : clean_movie_title(row))    ## clean row
merged_imdb["release_date"] = merged_imdb["release_date"].apply(lambda x : str(x).split("-")[0]) ## format data for merge
## a bit more cleaning on title
merged_imdb= merged_imdb[~merged_imdb["title"].str.startswith("(")] 
df_imdb = merged_imdb[merged_imdb["title"].str.contains("(", regex=False)]
df_imdb["title"] = df_imdb["title"].apply(lambda x: re.search(r'\((.*?)\)', x).group(1).strip() if re.search(r'\((.*?)\)', x) else x.strip())
merged_imdb= merged_imdb.append(df_imdb)
## make sure there are no duplicates
merged_imdb = merged_imdb.drop_duplicates()
## merge movieset data points together with movie data from movielens
merged_data =movies.merge( merged_imdb,on=["title", "release_date"])
## drop a bunch of columns i didnt use in the end
cols = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,26,29,30,32,33,35,41,42,43]
merged_data.drop(merged_data.columns[cols],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Next part of code is now vectorizing the keywords, production companies,cast and crew so we can pass these through a network normalized 

First I need to clean up the code by deleting all the columns I don't want and one hot encoding my genre column

In [26]:
import json
import ast

from ast import literal_eval

## retrive director for all the movies

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


## didnt use this in the end - was gonna add in the weighted rating
def weighted_rating(x):
    vote_counts = merged_data[merged_data['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = merged_data[merged_data['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.75)
    v = x['vote_count']+1 # added +1 - Dan
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)


## to create the soup i need needed to do a lot of formatting on the columns i wanted to vectorize
def create_dataset(df):

  ## get the cast and reformat them to "cast1, cast2, cast3"
  df["cast"] = df["cast"].apply(ast.literal_eval)
  df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
  df['cast'] = df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
  df['cast'] = df['cast'].apply(lambda x: ",".join(x))
  
  ## use genres from movielens 
  df['genres'] = movielens['Genre']

  # reformat keywords like cast
  df['keywords'] = df['keywords'].apply(literal_eval)
  df['keywords'] = df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
  df['keywords'] = df['keywords'].apply(lambda x: ",".join(x))

  ### reformat production companies 
  df['production_companies'] = df['production_companies'].apply(ast.literal_eval)
  df['production_companies'] = df['production_companies'].fillna("[]").apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
  df['production_companies'] = df['production_companies'].apply(lambda x: ",".join(x))

  ## reformate production countries
  df['production_countries'] = df['production_countries'].fillna('[]').apply(ast.literal_eval)
  df['production_countries'] = df['production_countries'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
  df['production_countries'] = df['production_countries'].apply(lambda x: ",".join(x))
  
  ## reformate spoken languages
  df['spoken_languages'] = df['spoken_languages'].fillna('[]').apply(ast.literal_eval).apply(lambda x: len(x) if isinstance(x, list) else np.nan)

  ## clean up crew
  df['crew'] = df['crew'].apply(literal_eval)
  ## get director
  df['director'] = df['crew'].apply(get_director) 
  

### this data wasnt used in the model in the end
  df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
  df['budget'] = df['budget'].replace(0, np.nan)
  df['revenue'] = df['revenue'].replace(0, np.nan)
  df['weighted_rating'] = df.apply(weighted_rating, axis=1)
  df['adult'].replace(['False','True'],[0,1],inplace=True)

In [27]:
create_dataset(merged_data)
## ended up not using this data
merged_data.drop(columns=['budget'],inplace=True)
merged_data.drop(columns=['revenue'],inplace=True)
merged_data.drop(columns=['vote_average'],inplace=True)
merged_data.drop(columns=['vote_count'],inplace=True)
merged_data.drop(columns=['crew'],inplace=True)

In [28]:
d_col = [1,4,6,7,10,11]
## create soup using all new columns created
merged_data['soup'] =  merged_data[merged_data.columns[d_col]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [29]:
## movieset data and movielens data merged on movieId, works nicely as the movie information from movielens was already merge
## using title so "merged_data already has the movieId"
works =  merged_data.merge(df,on='movieId')
## create the main dataframe which will be used for model
the_main_df = works[['movieId','userId','rating']].copy()
## create df used for creating vectorizaed matrix of soup used for model
soup_df = merged_data[['movieId','soup']].copy()
soup_df =soup_df.set_index('movieId')

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
## create token pattern to find english words in soup
token_pattern='(?u)[a-zA-Z][a-z ]+'
## create count vertorize matrix for model
tdidf =  TfidfVectorizer(stop_words='english',token_pattern=token_pattern)#, ngram_range=(3, 3))
matrix = tdidf.fit_transform(soup_df['soup'])

In [31]:
## Use mapping to map vectorize soup id to movie id for model
mapping = {id:i for i, id in enumerate(soup_df.index)}

## create test and train dataset
n = 10000
df_model = the_main_df.sample(frac=1).reset_index(drop=True)
df_train = df_model[:70000]
df_test = df_model[-n:]


## add the vectorized soup using the mapping created above, we need the trainset and test set to contain the right vectorized soup

train_count = []
# Iterate over all movie-ids and save the tfidf-vector
for id in df_train['movieId'].values:
    index = mapping[id]
    train_count.append(matrix[index])

test_count = []
# Iterate over all movie-ids and save the tfidf-vector
for id in df_test['movieId'].values:
    index = mapping[id]
    test_count.append(matrix[index])

In [32]:
## stack the vectorized rows vertically  to work with model
from scipy.sparse import vstack
train_tfidf = vstack(train_count)
test_tfidf = vstack(test_count)
test_tfidf  = test_tfidf.toarray()
train_tfidf = train_tfidf.toarray()

#Hybrid model

In [33]:
def rmse_metric(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))


## chose embedding hyperparameters
user_embed = 10
movie_embed = 10


## create hybrid model, this will include the soup data made above
user_id_input = Input(shape=[1], name='user')
movie_id_input = Input(shape=[1], name='movie')
tfidf_input = Input(shape=[8213], name='tfidf', sparse=True)


##embed user id
user_embedding = Embedding(output_dim=user_embed,
                           input_dim=944,
                           input_length=1,
                        name='user_embedding')(user_id_input)

# embed movie id                        
movie_embedding = Embedding(output_dim=movie_embed,
                            input_dim=2000,
                            input_length=1,
                            name='movie_embedding')(movie_id_input)

# concatenate matrix vectors with embedded features
tfidf_vectors = Dense(64, activation='softmax')(tfidf_input)
tfidf_vectors = Dense(32, activation='softmax')(tfidf_vectors)
user_vectors = Reshape([user_embed])(user_embedding)
movie_vectors = Reshape([movie_embed])(movie_embedding)
both = Concatenate()([user_vectors, movie_vectors, tfidf_vectors])
dense = Dense(512, activation='softmax')(both)
dense = Dropout(0.2)(dense)
output = Dense(1)(dense)


model = Model(inputs=[user_id_input, movie_id_input, tfidf_input], outputs=output)
model.compile(loss='mse', optimizer='adam',metrics=[rmse_metric])


# Train and test the network
model.fit([df_train['userId'], df_train['movieId'], train_tfidf], df_train['rating'],
          batch_size=2048, 
          epochs=200,shuffle=True, validation_split=0.1)

y_pred = model.predict([df_test['userId'], df_test['movieId'], test_tfidf])
y_true = df_test['rating'].values

rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Hybrid Deep Learning: {:.4f} RMSE'.format(rmse))

Epoch 1/200
31/31 [==============================] - 3s 38ms/step - loss: 13.5510 - rmse_metric: 3.6811 - val_loss: 13.2668 - val_rmse_metric: 3.6477
Epoch 2/200
31/31 [==============================] - 1s 30ms/step - loss: 13.0824 - rmse_metric: 3.6169 - val_loss: 12.7956 - val_rmse_metric: 3.5824
Epoch 3/200
31/31 [==============================] - 1s 28ms/step - loss: 12.5700 - rmse_metric: 3.5453 - val_loss: 12.3050 - val_rmse_metric: 3.5132
Epoch 4/200
31/31 [==============================] - 1s 29ms/step - loss: 12.1258 - rmse_metric: 3.4821 - val_loss: 11.7922 - val_rmse_metric: 3.4393
Epoch 5/200
31/31 [==============================] - 1s 30ms/step - loss: 11.6167 - rmse_metric: 3.4082 - val_loss: 11.2630 - val_rmse_metric: 3.3613
Epoch 6/200
31/31 [==============================] - 1s 30ms/step - loss: 11.0877 - rmse_metric: 3.3297 - val_loss: 10.7291 - val_rmse_metric: 3.2808
Epoch 7/200
31/31 [==============================] - 1s 28ms/step - loss: 10.5317 - rmse_metric: 3.2

RMSE score of 0.9150

# Test on U1 test data

To do this we first need to add all our extra features we made for u1 base

In [34]:
u1_merged =  merged_data.merge(u1_test,on='movieId')
u1_df = u1_merged[['movieId','userId','rating']].copy()

In [35]:
u1_test = []
# Iterate over all movie-ids and save the tfidf-vector
for id in u1_df['movieId'].values:
    index = mapping[id]
    u1_test.append(matrix[index])

u1_test_stack = vstack(u1_test)
u1_test_stack = u1_test_stack.toarray()

In [36]:
y_pred = model.predict([u1_df['userId'], u1_df['movieId'], u1_test_stack])
y_true = u1_df['rating'].values
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))

In [37]:
rmse

0.9480708868120268

U1 test gets a score of 0.9480708868120268 RMSE